In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score ,mean_squared_error , accuracy_score
from sklearn.model_selection import train_test_split , GridSearchCV 
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor,GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from tensorflow import keras

import joblib
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Dense, Dropout
# import seaborn as sns



In [2]:
data = pd.read_csv('../data/data.csv')

In [3]:
data.head()

,month,season,lat,lon,chl_df_a,sst,Species Category,Scientific Name,English Name,Local Name,Fish Temp In Category
0,2010-04-05,1,22.517914,67.195389,0.6,27.6,14,35,29,85,1
1,2010-04-05,1,22.517914,67.195389,0.6,27.6,5,11,16,89,1
2,2010-04-05,1,22.517914,67.195389,0.6,27.6,12,102,108,63,2
3,2010-04-05,1,22.517914,67.195389,0.6,27.6,12,79,31,77,2
4,2010-04-05,1,22.517914,67.195389,0.6,27.6,12,80,10,76,2


In [4]:
data.drop(['month','Species Category','chl_df_a', 'sst','Scientific Name','Local Name'],axis=1,inplace=True)
# data.drop(['month','Species Category','chl_df_a', 'sst','Scientific Name','English Name'],axis=1,inplace=True)

In [5]:
   # Perform any necessary data cleaning and preprocessing here

# Step 2: Encode categorical variables
# cat_vars = ['Local Name','Fish Temp In Category']
cat_vars = ['English Name','Fish Temp In Category']
label_encoders = {}
for var in cat_vars:
    label_encoders[var] = LabelEncoder()
    data[var] = label_encoders[var].fit_transform(data[var])


In [6]:
data.columns

Index(['season', 'lat', 'lon', 'English Name', 'Fish Temp In Category'], dtype='object')

In [7]:
# split the data into training and testing sets
# X = data[['season', 'Local Name', 'Fish Temp In Category']]
X = data[['season', 'English Name', 'Fish Temp In Category']]
y = data[['lat', 'lon']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [8]:
# normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Random Forest

In [9]:

# Create and train the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the model on the testing set
predictions = rf_model.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
print("Random Forest Regression MSE:", rmse)



Random Forest Regression MSE: 0.369195705716941


In [20]:

# # Visualize actual and predicted values for latitude
# plt.figure(figsize=(12, 6))
# plt.plot(y_test, label='Actual')
# plt.plot(predictions, label='Predicted')
# plt.xlabel('Sample')
# plt.ylabel('Latitude')
# plt.title('Actual vs Predicted Latitude')
# plt.legend()
# plt.show()

In [10]:
# save model
joblib.dump(rf_model, "../models/update_RF_model_eng.pkl")

['../models/update_RF_model_eng.pkl']

In [9]:
rf_model = joblib.load("../models/update_RF_model.pkl")
# Make predictions on new data
new_data = pd.DataFrame({
    'season': [2, 3, 1],
    'English Name': [85, 60, 70],
    "Fish Temp In Category":[2,1,1]
})



input_data = scaler.transform(new_data)

predictions = rf_model.predict(input_data)
print(predictions)

[[22.49715204 67.21624868]
 [21.52013588 68.19796753]
 [21.22911835 68.49038696]]


[[22.5166458  67.1966628 ]
 [21.82982084 67.88679006]
 [21.12070416 68.5993228 ]]

# LSTM and ANN

In [10]:
# reshape the data for the RNN model
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [11]:
# define the RNN model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=2))

In [12]:
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
# train the model with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10)
model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop])


Epoch 1/25
10833/10833 [==============================] - 73s 6ms/step - loss: 101.7344 - val_loss: 0.6332
Epoch 2/25
10833/10833 [==============================] - 57s 5ms/step - loss: 11.3049 - val_loss: 0.4815
Epoch 3/25
10833/10833 [==============================] - 62s 6ms/step - loss: 9.8041 - val_loss: 0.4847
Epoch 4/25
10833/10833 [==============================] - 59s 5ms/step - loss: 8.5163 - val_loss: 0.4638
Epoch 5/25
10833/10833 [==============================] - 60s 5ms/step - loss: 7.3622 - val_loss: 0.4146
Epoch 6/25
10833/10833 [==============================] - 54s 5ms/step - loss: 6.3142 - val_loss: 0.3903
Epoch 7/25
10833/10833 [==============================] - 54s 5ms/step - loss: 5.3868 - val_loss: 0.4475
Epoch 8/25
10833/10833 [==============================] - 54s 5ms/step - loss: 4.5362 - val_loss: 0.4108
Epoch 9/25
10833/10833 [==============================] - 62s 6ms/step - loss: 3.8016 - val_loss: 0.3490
Epoch 10/25
10833/10833 [===========================

KeyboardInterrupt: 

In [ ]:
# evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)

2408/2408 [==============================] - 14s 6ms/step - loss: 0.2402


In [ ]:
test_loss

0.24017511308193207

In [ ]:
model.save('../models/update_LSTM_model_eng.h5')